In [1]:
!pip install transformers[torch,sentencepiece] langchain langchain-community langchain-huggingface faiss-cpu jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

HUGGINGFACE_TOKEN = userdata.get('HF_TOKEN_READ')
!huggingface-cli login --token $HUGGINGFACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `LLM_Course` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LLM_Course`


In [3]:
from transformers.trainer_utils import set_seed

set_seed(42)

### Chat Modelの作成

In [20]:
import torch
from langchain_huggingface import (
    ChatHuggingFace,
    HuggingFacePipeline
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)

model_name = "hiroki-rad/Swallow-7b-hf-oasst1-21k-ja-aio-retriever"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--hiroki-rad--Swallow-7b-hf-oasst1-21k-ja-aio-retriever/snapshots/d1838745413c0ecf9387b89e08313cef24b91c5d/config.json
Model config LlamaConfig {
  "_name_or_path": "hiroki-rad/Swallow-7b-hf-oasst1-21k-ja-aio-retriever",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "max_sequence_length": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.1",
  "use_c

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--hiroki-rad--Swallow-7b-hf-oasst1-21k-ja-aio-retriever/snapshots/d1838745413c0ecf9387b89e08313cef24b91c5d/model.safetensors.index.json


model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.38M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--hiroki-rad--Swallow-7b-hf-oasst1-21k-ja-aio-retriever/snapshots/d1838745413c0ecf9387b89e08313cef24b91c5d/tokenizer.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--hiroki-rad--Swallow-7b-hf-oasst1-21k-ja-aio-retriever/snapshots/d1838745413c0ecf9387b89e08313cef24b91c5d/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--hiroki-rad--Swallow-7b-hf-oasst1-21k-ja-aio-retriever/snapshots/d1838745413c0ecf9387b89e08313cef24b91c5d/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--hiroki-rad--Swallow-7b-hf-oasst1-21k-ja-aio-retriever/snapshots/d1838745413c0ecf9387b89e08313cef24b91c5d/tokenizer_config.json
loading file chat_template.jinja from cache at None


In [21]:
generation_config = {
    "max_new_tokens": 32,
    "do_sample": False,
    "temperature": None,
    "top_p": None
}

# テキスト生成を行うパイプラインを作成
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    **generation_config
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

chat_model = ChatHuggingFace(llm=llm, tokenizer=tokenizer)

Device set to use cpu


### Embedding Modelの作成

In [7]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedding_model_name = "BAAI/bge-m3"

embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"model_kwargs": {"torch_dtype": torch.float16}}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

loading file sentencepiece.bpe.model from cache at /root/.cache/huggingface/hub/models--BAAI--bge-m3/snapshots/5617a9f61b028005a4858fdac845db406aefb181/sentencepiece.bpe.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--BAAI--bge-m3/snapshots/5617a9f61b028005a4858fdac845db406aefb181/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--BAAI--bge-m3/snapshots/5617a9f61b028005a4858fdac845db406aefb181/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--BAAI--bge-m3/snapshots/5617a9f61b028005a4858fdac845db406aefb181/tokenizer_config.json
loading file chat_template.jinja from cache at None


model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### データストアの構築

In [8]:
# 検索対象の文書集合のファイルをダウンロード
!wget  \
https://github.com/ghmagazine/llm-book/raw/main/chapter13/docs.json

--2025-01-13 01:39:28--  https://github.com/ghmagazine/llm-book/raw/main/chapter13/docs.json
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ghmagazine/llm-book/main/chapter13/docs.json [following]
--2025-01-13 01:39:28--  https://raw.githubusercontent.com/ghmagazine/llm-book/main/chapter13/docs.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1178382 (1.1M) [text/plain]
Saving to: ‘docs.json’

docs.json           100%[===================>]   1.12M  --.-KB/s    in 0.02s   

2025-01-13 01:39:28 (51.2 MB/s) - ‘docs.json’ saved [1178382/1178382]



In [9]:
from langchain_community.document_loaders import JSONLoader

# JSONファイルから文書を読み込むためのDocument Loaderを初期化
document_loader = JSONLoader(
    file_path="./docs.json",  # 読み込みを行うファイル
    jq_schema=".text",  # 読み込み対象のフィールド
    json_lines=True,  # JSON Lines形式のファイルであることを指定
)

# 文書の読み込みを実行
documents = document_loader.load()

# 読み込まれた文書数を確認
print(len(documents))


103


In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 文書を指定した文字数で分割するText Splitterを初期化
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # 分割する最大文字数
    chunk_overlap=100,  # 分割された文書間で重複させる最大文字数
    add_start_index=True,  # 元の文書における開始位置の情報を付与
)

# 文書の分割を実行
split_documents = text_splitter.split_documents(documents)

# 分割後の文書数を確認
print(len(split_documents))

### 検索対象の文書のベクトルインデックスの作成

In [11]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(split_documents, embedding_model)

print(vectorstore.index.ntotal)

1475


### Retrieverコンポーネントの作成

In [12]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [13]:
from pprint import pprint

retrieved_documents = retriever.invoke("四国地方で一番高い山は？")

pprint(retrieved_documents)

[Document(id='a276d16c-dbe0-483b-a8a2-197f82b9a9e9', metadata={'source': '/content/docs.json', 'seq_num': 26, 'start_index': 0}, page_content='この項目に含まれる文字「鎚」は、オペレーティングシステムやブラウザなどの環境により表示が異なります。 石鎚山（いしづちさん、いしづちやま）は、四国山地西部に位置する標高1,982 mの山で、近畿以西を「西日本」とした場合の西日本最高峰で、山頂から望む展望が四国八十八景64番に選定。愛媛県西条市と久万高原町の境界に位置する。 石鉄山、石鈇山、石土山、石槌山とも表記され、伊予の高嶺とも呼ばれる。『日本霊異記』には「石槌山」と記され、延喜式の神名帳（延喜式神名帳）では「石鉄神社」と記されている。前神寺および横峰寺では「石鈇山（しゃくまざん）」とも呼ぶ。'),
 Document(id='490a7d9d-56b0-4c11-9cce-16efe243b633', metadata={'source': '/content/docs.json', 'seq_num': 1, 'start_index': 0}, page_content='富士山（ふじさん）は、静岡県（富士宮市、富士市、裾野市、御殿場市、駿東郡小山町）と山梨県（富士吉田市、南都留郡鳴沢村）に跨る活火山である。標高3776.12 m、日本最高峰（剣ヶ峰）の独立峰で、その優美な風貌は日本国外でも日本の象徴として広く知られている。 数多くの芸術作品の題材とされ芸術面のみならず、気候や地層など地質学的にも社会に大きな影響を与えている。懸垂曲線の山容を有した玄武岩質成層火山で構成され、その山体は駿河湾の海岸まで及ぶ。'),
 Document(id='7d09cbf7-a564-437f-b5c0-bfdfa979061d', metadata={'source': '/content/docs.json', 'seq_num': 96, 'start_index': 0}, page_content='四阿山（あずまやさん）は、長野県と群馬県の県境に跨る山。標高2,354 m。日本百名山の一つに数えられている。吾妻山・吾嬬山（あがつま

### RAGのChainの構築と実行

In [14]:
from langchain_core.prompts import ChatPromptTemplate

# 任意のqueryからメッセージを構築するPrompt Templateを作成
rag_prompt_text = (
    "あなたには今からクイズに答えてもらいます。"
    "問題を与えますので、その解答のみを簡潔に出力してください。\n"
    "また解答の参考になりうるテキストを与えます。"
    "解答を含まない場合もあるのでその場合は無視してください。\n\n"
    "---\n{context}\n---\n\n問題: {query}"
)
rag_prompt_template = ChatPromptTemplate.from_messages(
    [("user", rag_prompt_text)]
)

In [15]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

def format_documents_func(documents: list[Document]) -> str:
    """文書のリストを改行で連結した一つの文字列として返す"""
    return "\n\n".join(
        document.page_content for document in documents
    )

format_documents = RunnableLambda(format_documents_func)

In [22]:
from langchain_core.prompt_values import ChatPromptValue

def chat_model_resp_only_func(
        chat_prompt_value: ChatPromptValue
) -> str:
    """chat_modelにchat_prompt_valueを入力し
    出力からモデルの応答部分のみを文字列で返す"""
    chat_prompt = chat_model._to_chat_prompt(
        chat_prompt_value.messages
    )
    chat_output_message = chat_model.invoke(chat_prompt_value)
    response_text = chat_output_message.content[len(chat_prompt):]
    return response_text

chat_model_resp_only = RunnableLambda(chat_model_resp_only_func)

In [23]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever | format_documents,
        "query": RunnablePassthrough(),
    }
    | rag_prompt_template
    | chat_model_resp_only
)

In [24]:
# Chainを実行し、結果を確認
rag_chain_output = rag_chain.invoke("四国地方で一番高い山は？")
print(rag_chain_output)

石鎚山
